In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

22/07/23 15:55:27 WARN Utils: Your hostname, kuchoco-GP62-6QE resolves to a loopback address: 127.0.1.1; using 192.168.219.113 instead (on interface wlp2s0)
22/07/23 15:55:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/23 15:55:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_green = spark.read.parquet('data/green/*')
df_green.registerTempTable('green')

In [6]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-19 00:52:30|  2019-12-19 00:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [17]:
df_green_revenue = spark.sql(
    """
    SELECT
        date_trunc('hour', lpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
        hour, zone 
    ORDER BY
        hour, zone
    """
)

In [18]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 03:00:00| 116|             22.63|             2|
|2020-01-01 03:00:00| 243|             15.29|             1|
|2020-01-01 05:00:00| 223|             36.96|             1|
|2020-01-01 07:00:00| 130|              11.8|             1|
|2020-01-01 08:00:00|  41|27.150000000000002|             3|
|2020-01-01 08:00:00|  42|             13.11|             1|
|2020-01-01 08:00:00|  75|              18.3|             1|
|2020-01-01 08:00:00|  82|              11.8|             1|
|2020-01-01 08:00:00| 116|              6.81|             1|
|2020-01-01 08:00:00| 130|             29.96|             3|
|2020-01-01 08:00:00| 168|              21.3|             1|
|2020-01-01 08:00:00| 179|               7.8|             1|
|2020-01-01 08:00:00| 248|              11.3|             1|
|2020-01-01 09:00:00|   

In [26]:
df_green_revenue\
    .repartition(20)\
    .write.parquet('data/report/green', mode='overwrite')

In [21]:
df_yelllow = spark.read.parquet('data/yellow/*')
df_yelllow.registerTempTable('yellow')

In [24]:
df_yellow_revenue = spark.sql(
    """
    SELECT
        date_trunc('hour', tpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM
        yellow
    WHERE
        tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
        hour, zone 
    ORDER BY
        hour, zone
    """
)

In [27]:
df_yellow_revenue\
    .repartition(20)\
    .write.parquet('data/report/yellow', mode='overwrite')

22/07/23 17:24:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
